---
title: "Spark SQL Job Data Analysis"
author: "Soham S Deshkhaire"
format: html
embed-resources: true
date: "2020-02-25"
date-format: long
execute:
  echo: true
---

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("JobPostingsAnalysis").getOrCreate()

df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine", "true").option("escape", "\"").csv("data/lightcast_job_postings.csv")
df.createOrReplaceTempView("jobs")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/26 03:27:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/26 03:27:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [2]:
# Display the first Ten rows
df.show(10)

# Show the schema (column names & data types)
df.printSchema()

+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+---------+--------------------+--------------------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+------------------+------+--------------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+-------------

In [3]:
#number of job postings for each employment type and order them in descending order.
job_counts_by_type = spark.sql("""
    SELECT EMPLOYMENT_TYPE_NAME, COUNT(*) AS job_count
    FROM jobs
    GROUP BY EMPLOYMENT_TYPE_NAME
    ORDER BY job_count DESC
    LIMIT 10
""")

# Show the result
job_counts_by_type.show()

+--------------------+---------+
|EMPLOYMENT_TYPE_NAME|job_count|
+--------------------+---------+
|Full-time (> 32 h...|    69176|
|Part-time (â‰¤ 32...|     2298|
|Part-time / full-...|      980|
|                NULL|       44|
+--------------------+---------+



In [4]:
#Question 1: How many job postings we have in the dataset?
total_jobs = spark.sql("""
    SELECT COUNT(ID) AS total_unique_job_postings
    FROM jobs
""")
total_jobs.show()

+-------------------------+
|total_unique_job_postings|
+-------------------------+
|                    72476|
+-------------------------+



In [5]:
top_5 = spark.sql("""
    SELECT TITLE_RAW AS title_Job, COUNT(*) AS job_count
    FROM jobs
    GROUP BY TITLE_RAW
    ORDER BY job_count DESC
    LIMIT 5
""")

top_5.show()

+--------------------+---------+
|           title_Job|job_count|
+--------------------+---------+
|        Data Analyst|     4201|
|Enterprise Architect|      808|
| Senior Data Analyst|      724|
|Business Intellig...|      686|
|        Data Modeler|      281|
+--------------------+---------+



In [6]:
avg_salary_by_type = spark.sql("""
    SELECT EMPLOYMENT_TYPE_NAME, AVG(SALARY) AS avg_salary
    FROM jobs
    WHERE SALARY IS NOT NULL AND EMPLOYMENT_TYPE IS NOT NULL
    GROUP BY EMPLOYMENT_TYPE_NAME
    ORDER BY avg_salary DESC
""")

avg_salary_by_type.show()

+--------------------+------------------+
|EMPLOYMENT_TYPE_NAME|        avg_salary|
+--------------------+------------------+
|Full-time (> 32 h...|118897.55860862407|
|Part-time / full-...| 105621.2423263328|
|Part-time (â‰¤ 32...| 98802.50963391137|
+--------------------+------------------+



In [7]:
#What five states have the most job postings
top_states = spark.sql("""
    SELECT STATE_NAME, COUNT(*) AS job_count
    FROM jobs
    WHERE STATE_NAME IS NOT NULL
    GROUP BY STATE_NAME
    ORDER BY job_count DESC
    LIMIT 5
""")

top_states.show()

+----------+---------+
|STATE_NAME|job_count|
+----------+---------+
|     Texas|     8067|
|California|     7084|
|   Florida|     3645|
|  Virginia|     3636|
|  Illinois|     3538|
+----------+---------+



In [8]:
#Calculate the salary range (max-min) for each job title in a California
california_data =  spark.sql(""" 
    SELECT TITLE_RAW as job_title, MAX(SALARY_TO)- MAX(SALARY_FROM) AS Diff_in_salary
    FROM jobs
    WHERE STATE_NAME = 'California'
    GROUP BY TITLE_RAW
    ORDER BY Diff_in_salary DESC
""")
california_data.show()

+--------------------+--------------+
|           job_title|Diff_in_salary|
+--------------------+--------------+
|San Diego GI Posi...|        784000|
|Analytics Enginee...|        550000|
|Analytics Enginee...|        550000|
|Oracle Utilities ...|        238300|
|Data Analytics Le...|        227000|
|ServiceNow Enterp...|        218300|
|Corporate Systems...|        215000|
|(USA) Senior Dist...|        208000|
|AI Portfolio & Ec...|        206100|
|Principal Enterpr...|        195600|
|Climate Data Analyst|        195000|
|Associate, F&S, S...|        190000|
|Security & Privac...|        186000|
|           Associate|        185000|
|Data Analyst, Sup...|        178000|
|  Data Analyst, Semi|        178000|
|Developer Archite...|        176700|
|Senior Director &...|        176000|
|Senior Network Ar...|        175500|
|    Client Architect|        174200|
+--------------------+--------------+
only showing top 20 rows


In [9]:
#What top 5 industries have the highest average salaries, and ,more than 100 job postings?
top_5_indus =  spark.sql(""" 
    SELECT NAICS_2022_6_NAME AS Industry, AVG(SALARY) AS Avg_salary, COUNT(*) AS job_count
    FROM jobs
    WHERE NAICS_2022_6_NAME IS NOT NULL AND SALARY IS NOT NULL
    GROUP BY NAICS_2022_6_NAME
    HAVING COUNT(*) >100
    ORDER BY Avg_salary DESC
    LIMIT 5
""")
top_5_indus.show()

+--------------------+------------------+---------+
|            Industry|        Avg_salary|job_count|
+--------------------+------------------+---------+
|Web Search Portal...|190195.80506329113|      395|
|Offices of Certif...| 162985.4912891986|      574|
|All Other Miscell...|150997.92523364487|      214|
|Research and Deve...|148788.00746268657|      134|
|Pharmaceutical Pr...|145708.55970149254|      134|
+--------------------+------------------+---------+

